In [ ]:
!pip install transformers datasets

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoFeatureExtractor, ResNetForImageClassification,AutoImageProcessor
import torch
from PIL import Image

In [ ]:
data=pd.read_csv("/content/drive/MyDrive/DataForTraining/DogEmotion/labels.csv")
data=data.reset_index(drop=True)
data['label']=data['label'].map({'sad':0,'angry':1,'relaxed':2,'happy':3})

dataset=Dataset.from_pandas(data)
dataset=dataset.remove_columns(['Unnamed: 0'])


In [ ]:
#without changing the size
def get_image_and_tensor(example):
  filename=example['filename']
  location="/content/drive/MyDrive/DataForTraining/DogEmotion/data/"
  image=Image.open(location+filename)
  del example['filename']
  return {'image':image}

dataset=dataset.map(get_image_and_tensor)


In [ ]:
dataset=dataset.train_test_split(test_size=0.2)

In [ ]:
from transformers import AutoImageProcessor

checkpoint = "google/vit-base-patch16-224-in21k"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize])

def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples

dataset = dataset.with_transform(transforms)

from transformers import DefaultDataCollator
data_collator = DefaultDataCollator()

In [ ]:
!pip install evaluate

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from huggingface_hub import login
login()

In [ ]:

id2label={0:'sad',1:'angry',2:'happy',3:'relaxed'}
label2id={'sad':0,'angry':1,'happy':2,'relaxed':3}

from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=4,
    id2label=id2label,
    label2id=label2id,
)

training_args = TrainingArguments(
    output_dir="Dewa/dog_emotion_v2",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)

trainer = Trainer(
    model=model.to('cuda'),
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
)

trainer.train()